# Adding ETPs
We now only have a list of stocks. But we might also want to trade SPY, TLT or VXX. I am only interested in a selection of ETPs and indices; these were found by taking the most liquid ETF in its category at [etfdb.com](etfdb.com).

* Size ETFs: "SPY", "IJH", "IJR", 
    * Large-cap, Mid-cap, Small-cap, 
* Bond ETFs: "BIL", "SHY", "IEF", "TLT", "JPST", "VCSH", "VCIT", "VCLT" 
    * risk-free (1-3M, 1-3Y, 7-10Y, 20+Y), corporate (Overnight, 1-5Y, 5-10Y, 10-25Y)
* Regions: "EEM", "IXUS", + SPY
* Region/Larger countries ETFs: "FXI", "EEM", "VGK", "EWY", "EWZ", "EWJ", "INDA", "EWW" + SPY
    * Based on the Finviz map the largest region ETFs.
* Country ETFs: 'EWA', 'EWO', 'EWK', 'EWC', 'EWQ', 'EWG', 'EWH', 'EWI', 'EWJ', 'EWM', 'EWW', 'EWN', 'EWS', 'EWP', 'EWD', 'EWL', 'EWU' + SPY
    * The iShares ETFs that were released before 2000 and thus have a very long history of data. Often the most liquid.
* Sector ETFs: "XLE", "XLU", "XLK", "XLB", "XLP", "XLY", "XLC", "XLV", "XLF", "XLRE"
    * The sectors that Finviz also uses.
* Commodity ETFs "GLD", "SLV"
    * Gold, silver
* Volatility ETPs: "VXX", "UVIX", "SVIX", "SVXY"
    * Long, Long 2X, Short 1x, Short 0.5x (1x before and including 2018-02-27)

*Note: ETPs is an umbrella term for ETFs, ETVs and ETNs. However most investors just refer to them as 'ETFs'.*

In [ ]:
#####
from polygon.rest import RESTClient
from datetime import date
from tickers import get_tickers
from times import first_trading_date_after_equal, last_trading_date_before_equal
import pandas as pd
import numpy as np

POLYGON_DATA_PATH = "../data/polygon/"
START_DATE = first_trading_date_after_equal(date(2003, 9, 10))
END_DATE = last_trading_date_before_equal(date(2024, 4, 19))

with open(POLYGON_DATA_PATH + "secret.txt") as f:
    KEY = next(f).strip()

client = RESTClient(api_key=KEY)

The ETFs can be added here manually.

In [2]:
size_ETFs = ["SPY", "IJH", "IJR"]
bond_ETFs = ["BIL", "SHY", "IEF", "TLT", "JPST", "VCSH", "VCIT", "VCLT" ]
sector_ETFs = ["XLE", "XLU", "XLK", "XLB", "XLP", "XLY", "XLC", "XLV", "XLF", "XLRE"]
region_ETFs = ["EEM", "IXUS"]

subregion_ETFs = ["FXI", "EEM", "VGK", "EWY", "EWZ", "EWJ", "INDA", "EWW"]
country_ETFs = ['EWA', 'EWO', 'EWK', 'EWC', 'EWQ', 'EWG', 'EWH', 'EWI', 'EWJ', \
    'EWM', 'EWW', 'EWN', 'EWS', 'EWP', 'EWD', 'EWL', 'EWU']
commodity_ETFs = ["GLD", "SLV"]
volatility_ETFs = ["VXX", "UVIX", "SVIX", "SVXY"]

all_ETFs = list(set(size_ETFs + bond_ETFs + sector_ETFs + region_ETFs + \
    country_ETFs + subregion_ETFs + commodity_ETFs + volatility_ETFs))

In [3]:
ticker_list_ETF = client.list_tickers(type="ETF", date=END_DATE.isoformat(), active=True, market='stocks', limit=1000)
ticker_list_ETN = client.list_tickers(type="ETN", date=END_DATE.isoformat(), active=True, market='stocks', limit=1000)
ticker_list_ETV = client.list_tickers(type="ETV", date=END_DATE.isoformat(), active=True, market='stocks', limit=1000)

ticker_list_ETF = pd.DataFrame(ticker_list_ETF)
ticker_list_ETN = pd.DataFrame(ticker_list_ETN)
ticker_list_ETV = pd.DataFrame(ticker_list_ETV)

tickers_ETP = pd.concat([ticker_list_ETF, ticker_list_ETN, ticker_list_ETV]).reset_index()
tickers_ETP = tickers_ETP[['ticker', 'name', 'active', 'cik', 'composite_figi', 'type']]
tickers_ETP = tickers_ETP[tickers_ETP["ticker"].isin(all_ETFs)]
tickers_ETP = tickers_ETP.reindex(columns = ["ID", "ticker", "name", "active", "start_date", "end_date", "type", "cik", "composite_figi"]) 
tickers_ETP = tickers_ETP[~tickers_ETP['ticker'].duplicated()]

tickers_ETP.reset_index(drop=True, inplace=True)

tickers_ETP.head(3)

,ID,ticker,name,active,start_date,end_date,type,cik,composite_figi
0,NaN,BIL,SPDR Bloomberg 1-3 Month T-Bill ETF,True,NaN,NaN,ETF,0001064642,BBG000RFQSH8
1,NaN,EEM,iShares MSCI Emerging Markets ETF,True,NaN,NaN,ETF,0000930667,BBG000M0P5L2
2,NaN,EWA,iShares MSCI Australia ETF,True,NaN,NaN,ETF,0000930667,BBG000BDNJ29


Here we fill in the blanks using the <code>get_tickers_events</code> function. We need the start date from the ticker, which may not always be equal to START_DATE. I assume no delistings.

In [ ]:
for index, row in tickers_ETP.copy().iterrows():
    if row["type"] == "ETF" or row["type"] == "ETN" or row["type"] == "ETV":
        try:
            tickers_ETP.iloc[index, tickers_ETP.columns.get_loc("start_date")] = max(date.fromisoformat(client.get_ticker_events(row["ticker"]).events[-1]['date']), START_DATE)
        except Exception as e:
            tickers_ETP.iloc[index, tickers_ETP.columns.get_loc("start_date")] = START_DATE
            print(row["ticker"])
            print(repr(e))

# Set remaining columns
tickers_ETP["end_date"].fillna(END_DATE, inplace=True)
tickers_ETP["ID"] = tickers_ETP["ticker"] + '-' + tickers_ETP["start_date"].astype(str)

We need to manually change VXX and UVIX start date. The <code>get_ticker_events</code> is incorrect.

In [5]:
tickers_ETP.loc[tickers_ETP['ticker'] == 'VXX', "start_date"] = max(date(2009, 1, 29), START_DATE)
tickers_ETP.loc[tickers_ETP['ticker'] == 'UVIX', "start_date"] = max(date(2022, 3, 30), START_DATE)

I will call everything an 'ETF' for simplicity.

In [6]:
tickers_ETP['type'] = 'ETF'

In [7]:
# Merge tickers_v2 and ETFs to get tickers_v3
tickers_v2 = get_tickers(v=2)
tickers_v3 = pd.concat([tickers_v2, tickers_ETP])
tickers_v3.reset_index(drop=True, inplace=True)
tickers_v3.to_csv("../data/tickers_v3.csv") 
tickers_v3.tail(3)

,ID,ticker,name,active,start_date,end_date,type,cik,composite_figi
18445,XLY-2003-09-10,XLY,Consumer Discretionary Select Sector SPDR Fund,True,2003-09-10,2024-04-19,ETF,0001064641,BBG000BJ1MF9
18446,GLD-2008-05-22,GLD,"SPDR Gold Trust, SPDR Gold Shares",True,2008-05-22,2024-04-19,ETF,0001222333,BBG000CRF6Q8
18447,SLV-2006-04-28,SLV,iShares Silver Trust,True,2006-04-28,2024-04-19,ETF,0001330568,BBG000NDCRW7


# 4.2 Updates
We simply copy/paste the ETPs from the old to the new ticker_v3.

In [8]:
old_tickers_v3 = get_tickers(3)
ETPs = tickers_v3[tickers_v3['type'] == 'ETF']
ETPs.loc[:, 'end_date'] = END_DATE # I assume that no ETFs got delisted/renamed.
pd.concat([get_tickers(2), ETPs]).reset_index(drop=True).to_csv("../data/tickers_v3.csv")